# Setup

## pipeline config

In [5]:
CONFIG_CONTENT = """
[nlp]
lang = "en"
pipeline = ["llm_ner", "llm_rel"]

[components]

[components.llm_ner]
factory="llm"

[components.llm_ner.model]
@llm_models = "spacy.Local.v1"

[components.llm_ner.task]
@llm_tasks = "spacy.NER.v3"
labels = ["PERSON", "ORG", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART", "DATE", "TIME", "PERCENT", "MONEY", "QUANTITY", "ORDINAL", "CARDINAL", "PET", "TECHNICAL_CONCEPT"]


[components.llm_rel]
factory = "llm"

[components.llm_rel.model]
@llm_models = "spacy.Local.v1"

[components.llm_rel.task]
@llm_tasks = "spacy.REL.v1"
labels = ["is a romantic partner of", "is employed by", "is pet owner of", "lives in", "works on", "is a friend of", "has duration", "is a relative of"]
"""


with open('config.cfg', 'w') as f:
    f.write(CONFIG_CONTENT)
    
    
DATA_SOURCE_DIR = '/datasets/data_pkl'


## install requirements

In [6]:
REQUIREMENTS_CONTENT="""
spacy-llm @ git+https://github.com/tombedor/spacy-llm.git@0dba69841d68e31e9360bcf47035166516aadeb2
torch==2.3.0
transformers==4.40.1
sentencepiece
pandas
"""

with open('requirements.txt', 'w') as f:
    f.write(REQUIREMENTS_CONTENT)

In [7]:
!pip install -r requirements.txt

  Cloning https://github.com/tombedor/spacy-llm.git (to revision 0dba69841d68e31e9360bcf47035166516aadeb2) to /private/var/folders/8x/lhh8gmz10ts3rv_qd0jk3tyc0000gn/T/pip-install-kyyu9tye/spacy-llm_e62d94a99af5487e9ac22ff90e0d0862
  Running command git clone --filter=blob:none --quiet https://github.com/tombedor/spacy-llm.git /private/var/folders/8x/lhh8gmz10ts3rv_qd0jk3tyc0000gn/T/pip-install-kyyu9tye/spacy-llm_e62d94a99af5487e9ac22ff90e0d0862
  Running command git rev-parse -q --verify 'sha^0dba69841d68e31e9360bcf47035166516aadeb2'
  Running command git fetch -q https://github.com/tombedor/spacy-llm.git 0dba69841d68e31e9360bcf47035166516aadeb2
  Resolved https://github.com/tombedor/spacy-llm.git to commit 0dba69841d68e31e9360bcf47035166516aadeb2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade 

# Init pipeline

In [1]:
import logging
import spacy_llm
from spacy_llm.util import assemble
# import outlines.models.openai
# resolve entities
# import outlines
# import outlines.models
# login(token=token)


# set log level to stream to STDOUT
spacy_llm.logger.addHandler(logging.StreamHandler())
spacy_llm.logger.setLevel(logging.DEBUG)

nlp = assemble("config.cfg")



/Users/tombedor/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RegistryError: [E893] Could not find function 'spacy.Local.v1' in function registry 'llm_models'. If you're using a custom function, make sure the code is available. If the function is provided by a third-party package, e.g. spacy-transformers, make sure the package is installed in your environment.

Available names: spacy.Ada.v1, spacy.Ada.v2, spacy.Anthropic.v1, spacy.Azure.v1, spacy.Azure.v2, spacy.Babbage.v1, spacy.Babbage.v2, spacy.Claude-1-0.v1, spacy.Claude-1-0.v2, spacy.Claude-1-2.v1, spacy.Claude-1-2.v2, spacy.Claude-1-3.v1, spacy.Claude-1-3.v2, spacy.Claude-1.v1, spacy.Claude-1.v2, spacy.Claude-2.v1, spacy.Claude-2.v2, spacy.Claude-instant-1-1.v1, spacy.Claude-instant-1-1.v2, spacy.Claude-instant-1.v1, spacy.Claude-instant-1.v2, spacy.Code-Davinci.v1, spacy.Code-Davinci.v2, spacy.Cohere.v1, spacy.Command.v1, spacy.Command.v2, spacy.Curie.v1, spacy.Curie.v2, spacy.Davinci.v1, spacy.Davinci.v2, spacy.Dolly.v1, spacy.Falcon.v1, spacy.GPT-3-5.v1, spacy.GPT-3-5.v2, spacy.GPT-3-5.v

In [3]:
import spacy

spacy.util.registry.get_registry_names()

['architectures',
 'augmenters',
 'batchers',
 'callbacks',
 'cli',
 'datasets',
 'displacy_colors',
 'factories',
 'initializers',
 'languages',
 'layers',
 'lemmatizers',
 'llm_misc',
 'llm_models',
 'llm_queries',
 'llm_tasks',
 'loggers',
 'lookups',
 'losses',
 'misc',
 'models',
 'ops',
 'optimizers',
 'readers',
 'schedules',
 'scorers',
 'tokenizers',
 'vectors']

### Perform NER and relations

In [ ]:
import os
from time import sleep
import pandas as pd
from tqdm import tqdm

# DATA_PATH = '/datasets/data_pkl/msgs.pkl'
DATA_PATH = 'msgs.pkl'

with open(DATA_PATH, 'rb') as f:
    docs_df = pd.read_pickle(f)
    
docs = docs_df[0].tolist()


entities_rows = []
relations_rows = []
for doc in tqdm(docs):
    enriched_doc = nlp(doc)
    # sleep(1) # sleep for openai
    ents = enriched_doc.ents
    for ent in ents:
        entities_rows.append({"name": ent.text, "label": ent.label_, "fact": doc})  # type: ignore
    for rel in enriched_doc._.rel:
        dep_name = ents[rel.dep].text
        dep_label = ents[rel.dep].label_
        dest_name = ents[rel.dest].text
        dest_label = ents[rel.dest].label_

        # ignore self relations
        if (dep_name, dep_label) == (dest_name, dest_label):
            continue

        # ignore dates
        if dep_label == "DATE" or dest_label == "DATE":
            continue

        relations_rows.append(
            {
                "dep_name": dep_name,
                "dep_label": dep_label,
                "dest_name": dest_name,
                "dest_label": dest_label,
                "rel": rel.relation,
                "fact": doc,
            }
        )


entities_df = pd.DataFrame(entities_rows)
relations_df = pd.DataFrame(relations_rows)

with open('entities.pkl', 'wb') as f:
    entities_df.to_pickle(f)
    
with open('relations.pkl', 'wb') as f:
    relations_df.to_pickle(f)

### Resolve inconsistencies

In [ ]:
# group facts from entities
grouped_entities_df = entities_df.groupby(["name", "label"]).agg({"fact": lambda x: list(x)}).reset_index()
grouped_relations_df = (
    relations_df.groupby(["dep_name", "dep_label", "dest_name", "dest_label", "rel"]).agg({"fact": lambda x: list(x)}).reset_index()
)


# in some passes, a subset of the labels should be discarded
# perhaps, characterize the convo by work, personal, etc and tailor retrieval accordingly

# clashing entity types
labels_grouped = grouped_entities_df.groupby(["name"]).agg({"label": lambda x: list(x), "fact": lambda x: [item for sublist in x for item in sublist]}).reset_index()
clashing_entity_labels = labels_grouped[labels_grouped["label"].apply(lambda x: len(x) > 1)]


rels_grouped = grouped_relations_df.groupby(["dep_name", "dest_name"]).agg({"rel": lambda x: list(x), "fact": lambda x: [item for sublist in x for item in sublist]}).reset_index()
clashing_rels = rels_grouped[rels_grouped["rel"].apply(lambda x: len(x) > 1)]

In [ ]:


# huggingface_hub.login(token=os.environ['HF_TOKEN'])
# model = outlines.models.openai("gpt-3.5-turbo") 
# model = outlines.models.openai("gpt-4-0613")
# model = outlines.models.transformers("mistralai/Mistral-7B-Instruct-v0.2", device='mps')
# model = outlines.models.llamacpp("TheBloke/phi-2-GGUF", "phi-2.Q4_K_M.gguf", device='mps')


def pick_winning_label(row: Series) -> str:
    print('processing a request after 5 sec delay')
    sleep(5)
    labels = row["label"]
    name = row["name"]
    facts = "\n".join(row["fact"])

    prompt = f"""You are an entity resolution assistant. 
    You must classify the entity with name = {name}
    
    Use both your inherent knowledge, and these facts derived from chat logs:
    {facts} 
    """

    generator = outlines.generate.choice(model, labels)
    answer = generator(prompt)
    print(f"{name}: Choices = {labels}. WINNER = {answer}")
    return answer


new_df = clashing_entity_labels.apply(pick_winning_label, axis=1)

